In [1]:
import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
output_path = 'D:/documents/1_projects/GreenDesign/data/output/'

In [3]:
df = pd.read_excel(output_path + 'result.xlsx', index_col='Unnamed: 0')

## <center> Подготовка данных к анализу

## <center> Группировка инфы из столбцов

На данном этапе остаётся большое количество столбцов, в которых небольшими порциями отражена информация, данную информацию логически нужно сгруппировать в меньшее количество столбцов.

В качестве исходного файла работаем с таблицей result

### <center> Свод информации о виде и расходе топлива

Нужно -  
* создать столбцы 'название_топлива' (прир газ, топливный газ)
* создать столбцы 'вид_топлива' (жидкое, газообразное, др..)
* создать столбец 'расход_топлива' (в соответсвующих ед изменерия)

* В столбцах с видами топлива пропуски (NaN) заменяем на пустые строки, т.к. с NaN нельзя производить операции сложения, а с количеством на нули
* создаем списки с уникальными наименованиями топлива по виду
* создаем функцию переименования под пум
* создаем пустые столбцы (вид, наименование, расход)
* проходим по каждой строке из таблицы, определяем по какому топливу наибольший расход - заносим это топливо
* 
* удаляем столбцы с видом и расходом топлива

In [4]:
# в столбцах с видом топлива заполняем nan пробелами
df['Жидкое топливо (Вид топлива)'] = df['Жидкое топливо (Вид топлива)'].fillna('')
df['Газообразное топливо (Вид топлива)'] = df['Газообразное топливо (Вид топлива)'].fillna('')
df['Твердое топливо (Вид топлива)'] = df['Твердое топливо (Вид топлива)'].fillna('')

# в столбцах с расходом топлива заполняем nan нулями
df['Газообразное топливо (Расход топлива, тыс. м3)'] = df['Газообразное топливо (Расход топлива, тыс. м3)'].fillna(0)
df['Жидкое топливо (Расход топлива, т)'] = df['Жидкое топливо (Расход топлива, т)'].fillna(0)
df['Твердое топливо (Расход топлива, т)'] = df['Твердое топливо (Расход топлива, т)'].fillna(0)

# создаем списки с уникальными названиями топлива
# газ
gas_fuels_list = sorted(list(df['Газообразное топливо (Вид топлива)'].apply(lambda x: x.lower().strip()).unique()))
if '' in gas_fuels_list:
    gas_fuels_list.remove('')
# жидк
liquid_fuels_list = sorted(list(df['Жидкое топливо (Вид топлива)'].apply(lambda x: x.lower().strip()).unique()))
if '' in gas_fuels_list:
    gas_fuels_list.remove('')
# твердое
solid_fuels_list = sorted(list(df['Твердое топливо (Вид топлива)'].apply(lambda x: x.lower().strip()).unique()))
if '' in gas_fuels_list:
    gas_fuels_list.remove('')


# функция переименования вида топлива под ПУМ
def fuels_to_pum(text):
    '''
    функция должна принимать на вход вид топлива, который изначально есть в столбцах с топливом
    в зависимости от того, к какому виду топлива относится полученный аргумент
    возвращается вид топлива и название топлива
    '''
    # газообразное топливо
    if text.lower().strip() in gas_fuels_list:
        if 'природ' in text:
            return 'газообразное топливо', 'природный газ'
        elif 'пнг' in text or 'попут' in text:
            return 'газообразное топливо', 'попутный нефтяной газ'
        else:
            return 'газообразное топливо', 'иное газообразное топливо'
    
    # жидкое топливо
    elif text.lower().strip() in liquid_fuels_list:
        if 'мазут' in text:
            return 'жидкое топливо', 'мазут'
        elif 'бензин' in text:
            return 'жидкое топливо', 'автобензины'
        elif 'дт' in text or 'диз':
            return 'жидкое топливо', 'дизельное топливо'
        elif 'нефть' in text:
            return 'жидкое топливо', 'нефть'
        elif 'конденс' in text or 'гк' in text:
            return 'жидкое топливо', 'газовый конденсат'
        else:
            return 'жидкое топливо', 'другие нефтепродукты'
        
    # твердое топливо
    elif text.lower().strip() in solid_fuels_list:
        if 'кокс' in text:
            return 'твердое топливо', 'кокс нефтяной'
        elif 'камен' in text:
            return 'твердое топливо', 'каменный уголь'
        elif 'буры' in text:
            return 'твердое топливо', 'бурый уголь'
        elif 'брикет' in text:
            return 'твердое топливо', 'брикеты угольные'
        else:
            return 'твердое топливо', 'другое твердое топливо'
    else:
        return np.NaN, np.NaN
        
        
# создаем пустые столбцы
df['вид_топлива'] = 'unknown'
df['название_топлива'] = 'unknown'
df['расход_топлива_т_год'] = 0

# для каждой строки с таблице, определяем топливо, у которого наибольший расход
for line in df.index:
    # если все равно нулю
    if max(df.loc[line, 'Газообразное топливо (Расход топлива, тыс. м3)'], 
           df.loc[line, 'Жидкое топливо (Расход топлива, т)'], 
           df.loc[line, 'Твердое топливо (Расход топлива, т)'],) == 0:
        df.loc[line, 'вид_топлива'] = np.NaN
        df.loc[line, 'название_топлива'] = np.NaN
        df.loc[line, 'расход_топлива_т_год'] = np.NaN
    
    # если максимум по газообразному
    elif max(df.loc[line, 'Газообразное топливо (Расход топлива, тыс. м3)'] * 0.657 / 1000, 
             df.loc[line, 'Жидкое топливо (Расход топлива, т)'], 
             df.loc[line, 'Твердое топливо (Расход топлива, т)'],) == df.loc[line, 'Газообразное топливо (Расход топлива, тыс. м3)'] * 0.657 / 1000:
        df.loc[line, 'вид_топлива'] = fuels_to_pum(df.loc[line, 'Газообразное топливо (Вид топлива)'])[0]
        df.loc[line, 'название_топлива'] = fuels_to_pum(df.loc[line, 'Газообразное топливо (Вид топлива)'])[1]
        df.loc[line, 'расход_топлива_т_год'] = df.loc[line, 'Газообразное топливо (Расход топлива, тыс. м3)'] * 0.657 / 1000
        
    # если максимум по жидкому
    elif max(df.loc[line, 'Газообразное топливо (Расход топлива, тыс. м3)'] * 0.657 / 1000, 
             df.loc[line, 'Жидкое топливо (Расход топлива, т)'], 
             df.loc[line, 'Твердое топливо (Расход топлива, т)'],) == df.loc[line, 'Жидкое топливо (Расход топлива, т)']:
        df.loc[line, 'вид_топлива'] = fuels_to_pum(df.loc[line, 'Жидкое топливо (Вид топлива)'])[0]
        df.loc[line, 'название_топлива'] = fuels_to_pum(df.loc[line, 'Жидкое топливо (Вид топлива)'])[1]
        df.loc[line, 'расход_топлива_т_год'] = df.loc[line, 'Жидкое топливо (Расход топлива, т)']
        
    # если максимум по твердому
    elif max(df.loc[line, 'Газообразное топливо (Расход топлива, тыс. м3)'] * 0.657 / 1000, 
             df.loc[line, 'Жидкое топливо (Расход топлива, т)'], 
             df.loc[line, 'Твердое топливо (Расход топлива, т)'],) == df.loc[line, 'Твердое топливо (Расход топлива, т)']:
        df.loc[line, 'вид_топлива'] = fuels_to_pum(df.loc[line, 'Твердое топливо (Вид топлива)'])[0]
        df.loc[line, 'название_топлива'] = fuels_to_pum(df.loc[line, 'Твердое топливо (Вид топлива)'])[1]
        df.loc[line, 'расход_топлива_т_год'] = df.loc[line, 'Твердое топливо (Расход топлива, т)']
        
# удаляем использованные столбцы 
df = df.drop([
    'Жидкое топливо (Вид топлива)',
    'Газообразное топливо (Вид топлива)',
    'Твердое топливо (Вид топлива)',
    'Газообразное топливо (Расход топлива, тыс. м3)',
    'Жидкое топливо (Расход топлива, т)',
    'Твердое топливо (Расход топлива, т)'
], axis=1)


### <center> Работа с количеством выбросов газов

#### <center> Выбросы СО2

##### Виды выбросов СО2

In [5]:
# пропуски в колонках с количеством выбросов газа заполняем нулями
for col in df.columns:
    if 'Выбросы СО2' in col:
        df[col] = df[col].fillna(0)

# создаем столбец с пустыми стоковыми значениями в ячейках
df['вид_выбросов'] = 'unknown'

# в зависимости от того, в каком столбце изнаачально стояло количество выбросов, определяем вид выброса по источнику
for row in df.index:
    # for col in df.columns:
    # if 'Выбросы СО2' in col:
    if df.loc[row, '1. Стационарное сжигание топлива (Выбросы СО2, т/год)'] != 0:
        df.loc[row, 'вид_выбросов'] = 'Стационарное сжигание топлива'
    elif df.loc[row, '2. Сжигание в факелах (Выбросы СО2, т/год)'] != 0:
        df.loc[row, 'вид_выбросов'] = 'Факельное сжигание'
    elif df.loc[row, '3. Фугитивные выбросы (Выбросы СО2, т/год)'] != 0:
        df.loc[row, 'вид_выбросов'] = 'Фугитивные выбросы'
    elif df.loc[row, '4. Нефтепереработка (Выбросы СО2, т/год)'] != 0:
        df.loc[row, 'вид_выбросов'] = 'Переработка'
    elif df.loc[row, '5. Производство кокса (Выбросы СО2, т/год)'] != 0:
        df.loc[row, 'вид_выбросов'] = 'Производство кокса'
    elif df.loc[row, '12. Нефтехимическое производство (Выбросы СО2, т/год)'] != 0:
        df.loc[row, 'вид_выбросов'] = 'Нефтехимия'
    elif df.loc[row, '17. Прочие промышленные процессы (Выбросы СО2, т/год)'] != 0:
        df.loc[row, 'вид_выбросов'] = 'Прочие промышленные процессы'
    elif df.loc[row, '20 Автомобильный транспорт (на балансе) (Выбросы СО2, т/год)'] != 0:
        df.loc[row, 'вид_выбросов'] = 'Транспорт'
    elif df.loc[row, '24 Полигоны (Выбросы СО2, т/год)'] != 0:
        df.loc[row, 'вид_выбросов'] = 'Фугитивные выбросы'

##### Количество выбросов СО2

In [6]:
# создаем столбец, заполняем нулями
df['количество_выбросов_со2_т_год'] = 0

# проверка на то, чтобы в этих столбцах были именно float значения
for col in df.columns[9:62]:
    df[col] = df[col].apply(lambda x: x if (type(x) is int) | (type(x) is float) else x.replace(',', '.'))
    df[col] = df[col].astype('float64')

# заполняем колонку с количеством выбросов, если в какой-то из изходный колонок есть инфа
for row in df.index:
    for col in df.columns:
        if 'Выбросы СО2' in col:
            df.loc[row, 'количество_выбросов_со2_т_год'] += df.loc[row, col]
            
# колонки с нулями переводим в nan
df['количество_выбросов_со2_т_год'] = df['количество_выбросов_со2_т_год'].apply(lambda x: x if x != 0 else np.NaN)

#### <center> Выбросы СН4

##### Виды выбросов СН4

In [7]:
# пропуски в колонках с количеством выбросов газа заполняем нулями
for col in df.columns:
    if 'Выбросы СН4' in col:
        df[col] = df[col].fillna(0)


# в зависимости от того, в каком столбце изнаачально стояло количество выбросов, определяем вид выброса по источнику
for row in df.index:
    # for col in df.columns:
    # if 'Выбросы СО2' in col:
    if df.loc[row, '2. Сжигание в факелах (Выбросы СН4, т/год)'] != 0:
        df.loc[row, 'вид_выбросов'] = 'Факельное сжигание'
    elif df.loc[row, '3. Фугитивные выбросы (Выбросы СН4, т/год)'] != 0:
        df.loc[row, 'вид_выбросов'] = 'Фугитивные выбросы'
    elif df.loc[row, '23 Выбросы от управления СВ (БОСы) (Выбросы СН4, т/год)'] != 0:
        df.loc[row, 'вид_выбросов'] = 'Выбросы от управления СВ (БОСы)'
    elif df.loc[row, '24 Полигоны (Выбросы СН4, т/год)'] != 0:
        df.loc[row, 'вид_выбросов'] = 'Фугитивные выбросы'

##### Количество выбросов СН4

In [8]:
# создаем столбец, заполняем нулями
df['количество_выбросов_сh4_т_год'] = 0

# заполняем колонку с количеством выбросов, если в какой-то из изходный колонок есть инфа
for row in df.index:
    for col in df.columns:
        if 'Выбросы СН4' in col:
            df.loc[row, 'количество_выбросов_сh4_т_год'] += df.loc[row, col]
            
# колонки с нулями переводим в nan
df['количество_выбросов_сh4_т_год'] = df['количество_выбросов_сh4_т_год'].apply(lambda x: x if x != 0 else np.NaN)

#### <center> Выброс ПГ с учётом ПГП

##### Виды выбросов ПГ с учётом ПГП

In [9]:
# пропуски в колонках с количеством выбросов ПГ заполняем нулями
for col in df.columns:
    if 'Выброс ПГ' in col:
        df[col] = df[col].fillna(0)

# в зависимости от того, в каком столбце изнаачально стояло количество выбросов, определяем вид выброса по источнику
for row in df.index:
    if df.loc[row, '1. Стационарное сжигание топлива (Выброс ПГ с учётом ПГП, тонн СО2-экв/год)'] != 0:
        df.loc[row, 'вид_выбросов'] = 'Стационарное сжигание топлива'
    elif df.loc[row, '2. Сжигание в факелах (Выброс ПГ с учётом ПГП, тонн СО2-экв/год)'] != 0:
        df.loc[row, 'вид_выбросов'] = 'Факельное сжигание'
    elif df.loc[row, '3. Фугитивные выбросы (Выброс ПГ с учётом ПГП, тонн СО2-экв/год)'] != 0:
        df.loc[row, 'вид_выбросов'] = 'Фугитивные выбросы'
    elif df.loc[row, '4. Нефтепереработка (Выброс ПГ с учётом ПГП, тонн СО2-экв/год)'] != 0:
        df.loc[row, 'вид_выбросов'] = 'Переработка'
    elif df.loc[row, '5. Производство кокса (Выброс ПГ с учётом ПГП, тонн СО2-экв/год)'] != 0:
        df.loc[row, 'вид_выбросов'] = 'Производство кокса'
    elif df.loc[row, '12. Нефтехимическое производство (Выброс ПГ с учётом ПГП, тонн СО2-экв/год)'] != 0:
        df.loc[row, 'вид_выбросов'] = 'Нефтехимия'
    elif df.loc[row, '17. Прочие промышленные процессы (Выброс ПГ с учётом ПГП, тонн СО2-экв/год)'] != 0:
        df.loc[row, 'вид_выбросов'] = 'Прочие промышленные процессы'
    elif df.loc[row, '20 Автомобильный транспорт (на балансе) (Выброс ПГ с учётом ПГП, тонн СО2-экв/год)'] != 0:
        df.loc[row, 'вид_выбросов'] = 'Транспорт'
    elif df.loc[row, '23 Выбросы от управления СВ (БОСы) (Выброс ПГ с учётом ПГП, тонн СО2-экв/год)'] != 0:
        df.loc[row, 'вид_выбросов'] = 'Выбросы от управления СВ (БОСы)'
    elif df.loc[row, '24 Полигоны (Выброс ПГ с учётом ПГП, тонн СО2-экв/год)'] != 0:
        df.loc[row, 'вид_выбросов'] = 'Фугитивные выбросы'  
    elif df.loc[row, 'Косвенные от потребления эл. энергии (Охват 2) (Выброс ПГ с учётом ПГП, тонн СО2-экв/год)'] != 0:
        df.loc[row, 'вид_выбросов'] = 'Косвенные от закупаемой э/энергии'  
    elif df.loc[row, 'Косвенные от потребления тепла (Охват 2) (Выброс ПГ с учётом ПГП, тонн СО2-экв/год)'] != 0:
        df.loc[row, 'вид_выбросов'] = 'Косвенные от закупаемой т/энергии'  

##### Количество выбросов ПГ с учётом ПГП

In [10]:
# создаем столбец, заполняем нулями
df['количество_выбросов_пг_с_пгп_т_год'] = 0

# заполняем колонку с количеством выбросов, если в какой-то из изходный колонок есть инфа
for row in df.index:
    for col in df.columns:
        if 'Выброс ПГ' in col:
            df.loc[row, 'количество_выбросов_пг_с_пгп_т_год'] += df.loc[row, col]
            
# колонки с нулями переводим в nan
df['количество_выбросов_пг_с_пгп_т_год'] = df['количество_выбросов_пг_с_пгп_т_год'].apply(lambda x: x if x != 0 else np.NaN)

In [11]:
df['тип_выбросов'] = 'unknown'
for line in df.index:
    if 'закуп' in df.loc[line, 'вид_выбросов']:
        df.loc[line, 'тип_выбросов'] = 'охват 2'
    else:
        df.loc[line, 'тип_выбросов'] = 'охват 1'

#### <center> Удаление схлопнувшихся столбцов, и столбцов с малым количеством данных

In [12]:
# удаляем те столбцы, инфу из которых ранее использовали
for col in df.columns:
    if 'Выбросы СО2' in col or 'Выбросы СН4' in col or 'Выброс ПГ' in col:
        df = df.drop([col], axis=1)

# Находим столбцы, где пропусков более 98% (на данный момент это 3 значени, 3 записанных строки)        
cols_null_percent = df.isnull().mean() * 100
cols_with_null = cols_null_percent[cols_null_percent>98].sort_values(ascending=False)

#задаем минимальный порог: вычисляем 3% от числа строк
thresh = df.shape[0]*0.03
#удаляем столбцы, в которых более 97% пропусков
df = df.dropna(how='any', thresh=thresh, axis=1)

#и ещё один непонятный столбец
df = df.drop(['Вклад по каждому источнику (%)'], axis=1)

In [13]:
# что если удалить все строки, где нет инфы о количестве выбросов (оставляем заполненные строки)
mask1 = df['количество_выбросов_со2_т_год'].notnull()
mask2 = df['количество_выбросов_сh4_т_год'].notnull()
mask3 = df['количество_выбросов_пг_с_пгп_т_год'].notnull()

df = df[mask1 | mask2 | mask3].reset_index(drop=True)

In [14]:
# создаем пустой столбец с видом оборудования
df['вид_оборудования'] = ''

# функция переименования
def rename_equip(equip):
    if 'печ' in equip.lower() or 'горн' in equip.lower():
        return 'печь'
    elif 'котл' in equip.lower() or 'котел' in equip.lower() or 'котёл' in equip.lower():
        return 'котел'
    elif 'тэц' in equip.lower():
        return 'выработка энергии'
    elif 'установк' in equip.lower() or 'гру' in equip.lower():
        return 'технологическое обрудование'
    elif 'факел' in equip.lower():
        return 'факельное сжигание'
    elif 'авто' in equip.lower():
        return 'автотранспорт'
    else:
        return 'прочее'
    
    # elif 'полигон' in equip.lower():
    #     return 'полигон'
    # elif ('свеча' in equip.lower()) or ('севеча' in equip.lower()):
    #     return 'свеча'
    # elif 'факел' in equip.lower():
    #     return 'факел'
    # elif 'резервуар' in equip.lower():
    #     return 'резервуары'
    # elif 'труба' in equip.lower():
    #     return 'труба'
    # elif 'газомотокомпрессор' in equip.lower():
    #     return 'газомотокомпрессор'
    # else:
    #     return equip.lower()
    # else:
    #     return 'остальное'


df['вид_оборудования'] = df['Оборудование/ процесс с выделением ПГ'].apply(rename_equip)
# list(df['вид_оборудования'].unique())
# Количество смотрим
df.groupby(by=['вид_оборудования'])['Наименование ОГ'].count().sort_values(ascending=False)

вид_оборудования
печь                           19
автотранспорт                  17
прочее                         17
котел                          14
технологическое обрудование    10
факельное сжигание             10
выработка энергии               1
Name: Наименование ОГ, dtype: int64

In [15]:
df_result_1 = df[['Наименование ОГ', 'Субъект РФ', 'Цех', 'Наименование площадки',
    'Объект предприятия', 'вид_оборудования', 'вид_выбросов', 'тип_выбросов',
    'название_топлива', 'вид_топлива', 'расход_топлива_т_год',
    'количество_выбросов_со2_т_год', 'количество_выбросов_сh4_т_год', 'количество_выбросов_пг_с_пгп_т_год'
    ]]

In [16]:
df_result_1.to_excel(output_path + 'result_prepare.xlsx', sheet_name='main')